In [148]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt

In [149]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/mollysprecher/.wdm/drivers/chromedriver/mac64/95.0.4638.69/chromedriver] found in cache


In [150]:
def scrape_all():
    #initiate healess driver for deployment 
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)
    
    news_title, news_p = mars_news(browser)
    
    #run scraping functions and store in dict 
    data = {
        "news_title": news_title, 
        "news_paragraph": news_p,
        "featured_image": featured_image(browser), 
        "facts": mars_facts(),
        "hemispheres": hemispheres(browser),
        "last_modified": dt.datetime.now()
    }
    
    browser.quit()
    return data

### Visit the NASA Mars News Site

In [151]:
def mars_news(browser):
    url = 'https://redplanetscience.com/'
    browser.visit(url)
    browser.is_element_present_by_css('div.list_text', wait_time=1)
    
    #convert browser html to soup object 
    html = browser.html
    news_soup = soup(html, 'html.parser')
    
    try: 
        slide_elem = news_soup.select_one('div.list_text')
        slide_elem.find('div', class_='content_title')
    
        #parent element to find first a tag and save
        news_title = slide_elem.find('div', class_='content_title').get_text()
    
        #parent element to find paragraph text 
        news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
        news_p
        
    except AttributeError:
        return None, None
    
    return news_title, news_p




### JPL Space Images Featured Image

In [152]:
def featured_image(browser):
    # Visit URL
    url = 'https://spaceimages-mars.com'
    browser.visit(url)
    
    #find and click full image 
    full_image_elem = browser.find_by_tag('button')[1]
    full_image_elem.click()
    
    #parse html w soup 
    html = browser.html
    img_soup = soup(html, 'html.parser')
    
    #try/except 
    try: 
        img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
        
    except AttributeError:
        return None 
    
    #base url for absolute 
    img_url = f'https://spaceimages-mars.com/{img_url_rel}'
    
    return img_url 

### Mars Facts

In [153]:
def mars_facts(): 
    try: 
        #use read to scrape into df 
        df = pd.read_html('https://galaxyfacts-mars.com')[0]
        
    except BaseException:
        return None 
    
    df.columns=['Description', 'Mars', 'Earth']
    df.set_index('Description', inplace=True)
        
    #convert to html 
    return df.to_html()
        


# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [154]:
# 1. Use browser to visit the URL 
def hemispheres(browser):
    url = 'https://marshemispheres.com/'
    browser.visit(url)
    
    html = browser.html
    hemi_soup = soup(html, 'html.parser')
    
    hemisphere_image_urls = []
    results = hemi_soup.find('div', class_='collapsible results')
    items = results.find_all('div', class_='item')
    
    for item in items:
        hemisphere = {}
        title = item.find('h3').text
        links = item.find('a', class_='product-item')['href']
        whole_url = f'https://marshemispheres.com/{links}'
        
        browser.visit(whole_url)
        html = browser.html
        hemi_soup = soup(html, 'html.parser')
        
        img_url = browser.links.find_by_partial_text('Sample')['href']
        
        hemisphere['img_url'] = img_url
        hemisphere['title'] = title
        hemisphere_image_urls.append(hemisphere)
        
        browser.back()
        
    return hemisphere_image_urls



In [155]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[]

In [156]:
# 5. Quit the browser
browser.quit()

In [157]:
if __name__ == "__main__":
    print(scrape_all())

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/mollysprecher/.wdm/drivers/chromedriver/mac64/95.0.4638.69/chromedriver] found in cache


{'news_title': "NASA's Briefcase-Size MarCO Satellite Picks Up Honors", 'news_paragraph': 'The twin spacecraft, the first of their kind to fly into deep space, earn a Laureate from Aviation Week & Space Technology.', 'featured_image': 'https://spaceimages-mars.com/image/featured/mars2.jpg', 'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n 